In [1]:
import json
from tqdm import tqdm
import numpy as np
import pandas as pd
import gzip
import pickle
import gc
from datasets import load_dataset
from evaluate import load
from metrics import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
[nltk_data] Downloading package punkt to /home/skunk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
['unieval', 'bertscorefree', 'bartcnnscore', 'bartscore_pegasus_cnndm', 'bartscore_pegasus_newsroom', 'bartscore_pegasus_xsum']

['unieval',
 'bertscorefree',
 'bartcnnscore',
 'bartscore_pegasus_cnndm',
 'bartscore_pegasus_newsroom',
 'bartscore_pegasus_xsum']

### Loading data

In [3]:
data = {}

SummEval


In [4]:
data['summeval']={}

In [5]:
file_path = '/llmthonskdir/metrics/data/cnndm/model_annotations.aligned.paired.jsonl'
with open(file_path, 'r') as json_file:
    json_list = list(json_file)

data_temp={}
for i in range(len(json_list)):
    data_temp[i] = json.loads(json_list[i])

In [6]:
data['summeval']['srcs'] = [data_temp[key]['text'] for key in set(data_temp)]
data['summeval']['hyps'] = [data_temp[key]['decoded'] for key in set(data_temp)]
data['summeval']['refs'] = [data_temp[key]['references'] for key in set(data_temp)]
data['summeval']['annot'] = {}
data['summeval']['annot']['coh'] = [pd.DataFrame(data_temp[key]['expert_annotations'])['coherence'].mean() for key in set(data_temp)]
data['summeval']['annot']['con'] = [pd.DataFrame(data_temp[key]['expert_annotations'])['consistency'].mean() for key in set(data_temp)]
data['summeval']['annot']['flu'] = [pd.DataFrame(data_temp[key]['expert_annotations'])['fluency'].mean() for key in set(data_temp)]
data['summeval']['annot']['rel'] = [pd.DataFrame(data_temp[key]['expert_annotations'])['relevance'].mean() for key in set(data_temp)]

Newsroom

In [7]:
data['newsroom']={}

In [8]:
with open('/llmthonskdir/metrics/data/BARTScore/SUM/Newsroom/data.pkl', 'rb') as f:
    data_temp = pickle.load(f)

In [9]:
data['newsroom']['srcs']=[]
data['newsroom']['hyps']=[]
data['newsroom']['refs']=[]
data['newsroom']['annot']={}
data['newsroom']['annot']['coh'] = []
data['newsroom']['annot']['info'] = []
data['newsroom']['annot']['flu'] = []
data['newsroom']['annot']['rel'] = []

for key in set(data_temp):
    for system in set(data_temp[key]['sys_summs']):     
        data['newsroom']['srcs'].append(data_temp[key]['src'])
        data['newsroom']['hyps'].append(data_temp[key]['sys_summs'][system]['sys_summ'])
        data['newsroom']['refs'].append(data_temp[key]['ref_summ'])
        data['newsroom']['annot']['coh'].append(data_temp[key]['sys_summs'][system]['scores']['coherence'])
        data['newsroom']['annot']['info'].append(data_temp[key]['sys_summs'][system]['scores']['informativeness'])
        data['newsroom']['annot']['flu'].append(data_temp[key]['sys_summs'][system]['scores']['fluency'])
        data['newsroom']['annot']['rel'].append(data_temp[key]['sys_summs'][system]['scores']['relevance'])

QAGS-CNN

In [10]:
data['qagscnn']={}

In [11]:
with open('/llmthonskdir/metrics/data/BARTScore/SUM/QAGS_CNN/data.pkl', 'rb') as f:
    data_temp = pickle.load(f)

In [12]:
data['qagscnn']['srcs']=[]
data['qagscnn']['hyps']=[]
data['qagscnn']['refs']=[]
data['qagscnn']['annot']={}
data['qagscnn']['annot']['fact'] = []

for key in set(data_temp):
    for system in set(data_temp[key]['sys_summs']):     
        data['qagscnn']['srcs'].append(data_temp[key]['src'])
        data['qagscnn']['hyps'].append(data_temp[key]['sys_summs'][system]['sys_summ'])
        data['qagscnn']['refs'].append(data_temp[key]['ref_summ'])
        data['qagscnn']['annot']['fact'].append(data_temp[key]['sys_summs'][system]['scores']['fact'])

QAGS-XSUM

In [13]:
data['qagsxsum']={}

In [14]:
with open('/llmthonskdir/metrics/data/BARTScore/SUM/QAGS_XSUM/data.pkl', 'rb') as f:
    data_temp = pickle.load(f)

In [15]:
data['qagsxsum']['srcs']=[]
data['qagsxsum']['hyps']=[]
data['qagsxsum']['refs']=[]
data['qagsxsum']['annot']={}
data['qagsxsum']['annot']['fact'] = []

for key in set(data_temp):
    for system in set(data_temp[key]['sys_summs']):     
        data['qagsxsum']['srcs'].append(data_temp[key]['src'])
        data['qagsxsum']['hyps'].append(data_temp[key]['sys_summs'][system]['sys_summ'])
        data['qagsxsum']['refs'].append(data_temp[key]['ref_summ'])
        data['qagsxsum']['annot']['fact'].append(data_temp[key]['sys_summs'][system]['scores']['fact'])

HallXSUM

In [16]:
data['hallxsum']={}

In [17]:
dataset = load_dataset("xsum")
xsum_data = pd.concat((pd.DataFrame(dataset['train']),
                       pd.DataFrame(dataset['validation']),
                       pd.DataFrame(dataset['test'])), axis=0)

In [18]:
data_temp = pd.read_csv('/llmthonskdir/metrics/data/xsum_hallucination_annotations/factuality_annotations_xsum_summaries.csv')
data_temp['fact'] = data_temp['is_factual'].apply(lambda x: 1 if x == 'yes' else 0)
data_temp = data_temp[['fact','bbcid','system','summary']].groupby(['bbcid','system','summary']).mean().reset_index()

In [19]:
data['hallxsum']['srcs']=[]
data['hallxsum']['hyps']=[]
data['hallxsum']['refs']=[]
data['hallxsum']['annot']={}
data['hallxsum']['annot']['fact'] = []

for i in tqdm(range(data_temp.shape[0])):
    row=xsum_data[np.array(xsum_data.id).astype(int)==data_temp.iloc[i]['bbcid']]
    
    data['hallxsum']['srcs'].append(row.iloc[0].document)
    data['hallxsum']['hyps'].append(data_temp.iloc[i]['summary'])
    data['hallxsum']['refs'].append(row.iloc[0].summary)
    data['hallxsum']['annot']['fact'].append(data_temp.iloc[i]['fact'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1869/1869 [00:35<00:00, 52.99it/s]


### Extracting metrics

In [20]:
[(type(data[key]['refs'][0]),key) for key in set(data)]

[(str, 'hallxsum'),
 (str, 'newsroom'),
 (str, 'qagscnn'),
 (str, 'qagsxsum'),
 (list, 'summeval')]

In [21]:
for key in set(data):
    if key=='summeval':
        pass
    else:
        data[key]['refs'] = [[r] for r in data[key]['refs']]

In [22]:
for key in set(data):
    data[key]['metrics'] = {}

BERTScore

In [23]:
%%time

bert_scorer = BERTScorerOurs()

for key in data.keys():
    data[key]['metrics']['bertscorefree'] = bert_scorer.score(data[key]['hyps'], [[s] for s in data[key]['srcs']], agg="mean", batch_size=16) 
    torch.cuda.empty_cache()

CPU times: user 2min 6s, sys: 32 s, total: 2min 38s
Wall time: 2min 42s


BARTScore

In [24]:
%%time

bartcnn_scorer = BARTScorer(checkpoint='facebook/bart-large-cnn')

for key in data.keys():
    print(key)
    data[key]['metrics']['bartcnnscore'] = bartcnn_scorer.score(data[key]['srcs'], data[key]['hyps'], batch_size=16) 
    torch.cuda.empty_cache()

summeval
newsroom
qagscnn
qagsxsum
hallxsum
CPU times: user 2min 33s, sys: 1.33 s, total: 2min 34s
Wall time: 2min 25s


In [25]:
%%time

checkpoints = ['google/pegasus-cnn_dailymail', 'google/pegasus-newsroom', 'google/pegasus-xsum'] #', 
names = {'google/pegasus-cnn_dailymail':'cnndm', 'google/pegasus-newsroom':'newsroom', 'google/pegasus-xsum':'xsum'}
max_lengths = {'google/pegasus-cnn_dailymail':1024, 'google/pegasus-newsroom':512, 'google/pegasus-xsum':512}

for checkpoint in checkpoints:
    scorer = PegasusScorer(device='cuda', max_length=max_lengths[checkpoint], checkpoint=checkpoint)

    for key in data.keys():
        data[key]['metrics']['bartscore_pegasus_'+names[checkpoint]] = scorer.score(data[key]['srcs'], data[key]['hyps'], batch_size=8) 
        torch.cuda.empty_cache()

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-newsroom and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

CPU times: user 8min 13s, sys: 9.68 s, total: 8min 23s
Wall time: 7min 37s


Unieval

In [26]:
%%time

unieval_scorer = UniEvalScorerOurs()

for key in data.keys():
    print(key)
    data[key]['metrics']['unieval'] = unieval_scorer.score(data[key]['srcs'], data[key]['hyps'], data[key]['refs'], agg="mean") 
    torch.cuda.empty_cache()
    gc.collect()

summeval


Evaluating coherence of 1600 samples !!!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:54<00:00,  1.74it/s]


Evaluating consistency of 1600 samples !!!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 624/624 [05:06<00:00,  2.03it/s]


...

Evaluating fluency of 1869 samples !!!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 237/237 [00:12<00:00, 18.90it/s]


Evaluating relevance of 1869 samples !!!


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 234/234 [00:17<00:00, 13.40it/s]


CPU times: user 18min 21s, sys: 12min 51s, total: 31min 13s
Wall time: 26min 43s


### Saving data

In [28]:
f = open("data/data.pkl","wb")
pickle.dump(data,f)
f.close()